# 📍 Geolocalizador de Negocios por Código Postal
Este notebook permite consultar negocios por código postal usando la API de Google Places y visualizar los resultados en un mapa interactivo con Folium.

## 🔧 Instalación de dependencias

In [7]:
!pip install pandas requests python-dotenv folium

## 🔑 Configurar clave de API

In [8]:

import os
from dotenv import load_dotenv

# Carga clave desde .env o asignación directa temporal
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY", "TU_API_KEY_AQUI")


## ⚙️ Funciones auxiliares

In [9]:

import requests
import pandas as pd
import time

def get_coordinates(postal_code, country='ES'):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {'address': f'{postal_code}, {country}', 'key': API_KEY}
    response = requests.get(url, params=params).json()
    if response['status'] != 'OK':
        raise ValueError(f"Error al obtener coordenadas para {postal_code}")
    location = response['results'][0]['geometry']['location']
    return location['lat'], location['lng']

def get_places(lat, lng, radius=2000, business_type=None):
    places = []
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{lat},{lng}',
        'radius': radius,
        'key': API_KEY,
        'type': business_type
    }

    while True:
        res = requests.get(url, params=params).json()
        places.extend(res.get('results', []))
        if 'next_page_token' in res:
            time.sleep(2)
            params['pagetoken'] = res['next_page_token']
        else:
            break

    return places

def clean_data(places, postal_code):
    data = []
    for p in places:
        location = p.get('geometry', {}).get('location', {})
        data.append({
            'nombre': p.get('name'),
            'direccion': p.get('vicinity'),
            'codigo_postal': postal_code,
            'puntuacion_media': p.get('rating'),
            'numero_reviews': p.get('user_ratings_total'),
            'tipo_negocio': ', '.join(p.get('types', [])),
            'latitud': location.get('lat'),
            'longitud': location.get('lng')
        })
    return pd.DataFrame(data)


## 📦 Consultar locales y exportar CSV

In [ ]:

codigos_postales = ['28004', '28005', '28012', '28013', '28014', '28015'] # Barrios distrito Centro, Madrid
tipo_negocio = None  # Ej. 'restaurant', 'store', etc.

todos_los_locales = pd.DataFrame()

for cp in codigos_postales:
    try:
        lat, lng = get_coordinates(cp)
        lugares = get_places(lat, lng, business_type=tipo_negocio)
        df = clean_data(lugares, cp)
        todos_los_locales = pd.concat([todos_los_locales, df], ignore_index=True)
        print(f"✅ {len(df)} locales encontrados en {cp}")
    except Exception as e:
        print(f"⚠️ Error con código postal {cp}: {e}")

# Si se especifica tipo de negocio, úsalo en el nombre del archivo
nombre_tipo = tipo_negocio if tipo_negocio else "todos"
output_file = f"locales_{nombre_tipo}.csv"

todos_los_locales.to_csv(output_file, index=False)
print(f"\n✅ Archivo '{output_file}' generado exitosamente.")

✅ 60 locales encontrados en 28004
✅ 60 locales encontrados en 28005
✅ 41 locales encontrados en 28012
✅ 60 locales encontrados en 28013
✅ 60 locales encontrados en 28014
✅ 60 locales encontrados en 28015

✅ Archivo 'locales.csv' generado exitosamente.


## 🗺️ Visualizar resultados en mapa interactivo

In [11]:

import folium

df = pd.read_csv("locales.csv")
df = df.dropna(subset=['latitud', 'longitud'])

mapa = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=13)

for _, row in df.iterrows():
    popup_text = f"""
    <b>{row['nombre']}</b><br>
    Dirección: {row['direccion']}<br>
    Rating: {row['puntuacion_media']} ({row['numero_reviews']} reseñas)<br>
    Tipo: {row['tipo_negocio']}
    """
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['nombre']
    ).add_to(mapa)

mapa.save("mapa_locales.html")
mapa
